In [17]:

import pandas as pd
import sqlalchemy
from dotenv import dotenv_values

Importing Data

In [30]:
listings = pd.read_excel(".//Augmented Data//listings.xlsx", sheet_name="listings")

In [31]:
reviews = pd.read_excel(".//Augmented Data//reviews.xlsx")

In [32]:
hosts = pd.read_excel(".//Augmented Data//host.xlsx")

Configurating MySQL Connection

In [12]:
env_vars = dotenv_values(".env")
driver_dialect = f"{env_vars["DIALECT"]}+{env_vars["DRIVER"]}"
user_pass = f"{env_vars["USERNAME"]}:{env_vars["PASSWORD"]}"
host_port = f"{env_vars["HOSTNAME"]}:{env_vars["PORTNUMBER"]}"
db_name = f"{env_vars["DATABASENAME"]}"
engine = sqlalchemy.create_engine(driver_dialect+"://"+user_pass+ "@"+host_port+"/"+db_name)

Cleaning the Data

In [69]:
def review_rename_col():
    """
    Returns a dictionary with the names of the columns in reviews 
    df I want renamed and their renamements
    """
    reviews_rename_dict = {"number_of_reviews": "total_num_reviews",
                        "number_of_reviews_ltm": "num_reviews_ltm"}
    reviews_rename_dict["number_of_reviews_ltm"] = "num_reviews_ltm"
    reviews_rename_dict["number_of_reviews_l30d"] = "num_reviews_l30d"
    reviews_rename_dict["review_scores_rating"] = "rating_score"
    reviews_rename_dict["review_scores_accuracy"] = "accuracy_score"
    reviews_rename_dict["review_scores_value"] = "value_score"
    reviews_rename_dict["review_scores_cleanliness"] = "cleanliness_score"
    reviews_rename_dict["review_scores_checkin"] = "checkin_score"
    reviews_rename_dict["review_scores_location"] = "location_score"
    reviews_rename_dict["review_scores_communication"] = "communication_score"

    return reviews_rename_dict
reviews.rename(columns= review_rename_col(), inplace=True)

In [58]:
def listing_rename_col():
    """
    Returns a dictionary with the names of the columns in listings 
    df I want renamed and their renamements
    """
    listing_rename_dict = {"accomodates":"num_can_accomodate"}
    listing_rename_dict["id"] = "listing_id"
    listing_rename_dict["neighbourhood_cleansed"] = "neighbourhood"
    listing_rename_dict["bathrooms"] = "num_bathrooms"
    listing_rename_dict["bedrooms"] = "num_bedrooms"
    listing_rename_dict["beds"] = "num_beds"
    

    return listing_rename_dict
listings.rename(columns= listing_rename_col(), inplace=True)

In [33]:
hosts.drop(columns=["calculated_host_listings_count",
                    'calculated_host_listings_count_entire_homes', 
                    'calculated_host_listings_count_private_rooms',
                    'calculated_host_listings_count_shared_rooms'], inplace=True)

In [36]:
listings.drop(columns=['listing_url', 'latitude', 'longitude', 'bathrooms_text',
                       'minimum_minimum_nights', 'maximum_minimum_nights',
                       'minimum_maximum_nights', 'maximum_maximum_nights',
                       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
                       'availability_30', 'availability_60', 'availability_90',
                       'availability_365', 'license', 'expiry_date'], inplace=True)


In [37]:
reviews.drop(columns=["last_review", "first_review"], inplace=True)

In [60]:
def transform_to_bool(df):
    """
    Returns a dataframe where t is True and f is renamed with False
    """
    return df.map(lambda x: x == "t")


In [61]:
hosts["host_identity_verified"] = transform_to_bool(hosts["host_identity_verified"])
hosts["host_is_superhost"] = transform_to_bool(hosts["host_is_superhost"])
hosts["host_has_profile_pic"] = transform_to_bool(hosts["host_has_profile_pic"])

In [62]:
listings["instant_bookable"] = transform_to_bool(listings["instant_bookable"])
listings["has_availability"] = transform_to_bool(listings["has_availability"])

In [64]:
hosts.head()

,host_id,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified
0,2319,Gail,2008-08-19,"Montréal, Canada",within an hour,1.0,0.70,False,Little Burgundy,4,7,True,True
1,39653,Miozoti,2009-09-19,"Montréal, Canada",within an hour,1.0,0.91,True,Le Plateau,2,5,True,True
2,40853,Lili,2009-09-23,"Montréal, Canada",within an hour,1.0,0.80,True,Notre-Dame-de-Grace,2,2,True,True
3,125031,Maryline,2010-05-14,"Montréal, Canada",within an hour,1.0,0.99,False,Downtown Montreal,2,2,True,True
4,133747,Caroline,2010-05-27,"Montréal, Canada",within an hour,1.0,0.48,True,NaN,3,3,True,True


Exporting Data to MySQL

In [73]:
listings.to_sql("listings", engine, if_exists="append", index=False)

9539

In [74]:
hosts.to_sql("hosts", engine, if_exists="append", index=False)

3944

In [75]:
reviews.to_sql("reviews", engine, if_exists="append", index=False)

9539